Read in the necessary libraries

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

Read in the necessary data

In [97]:
df_detailed_listings = pd.read_csv('./resource/raw_data/detailed_listings.csv')
df_detailed_reviews = pd.read_csv('./resource/raw_data/detailed_reviews.csv')
df_listings = pd.read_csv('./resource/raw_data/listings.csv')
df_reviews = pd.read_csv('./resource/raw_data/reviews.csv')

Overview of the data - comparing detailed listings df to reduced listings df

In [98]:
columns_detailed_listings = df_detailed_listings.columns
columns_listings = df_listings.columns

detailed_listings_exclusive_features = [x for x in columns_detailed_listings if x not in columns_listings]
listings_exclusive_features = [x for x in columns_listings if x not in columns_detailed_listings]

print(f'Features which are only in the detailed listings csv: \n{detailed_listings_exclusive_features}\n')
print(f'Features which are only in the reduces listings csv: \n{listings_exclusive_features}')

Features which are only in the detailed listings csv: 
['listing_url', 'scrape_id', 'last_scraped', 'source', 'description', 'neighborhood_overview', 'picture_url', 'host_url', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'property_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'calendar_last_scraped', 'number_of_reviews_l30d', 'first_review', 'review_

Apparently all columns from the reduced listings csv are included in the detailed listings csv, except for the column 'neighbourhood_group'. After looking through all column labels it is likely the information out of the 'neighbourhood_group' is also inlcuded in the detailed listings csv, but with a diffrent columnname.

In [99]:
df_listings.neighbourhood_group.value_counts()

Friedrichshain-Kreuzberg    3516
Mitte                       3426
Pankow                      2480
Neukölln                    2103
Charlottenburg-Wilm.        1287
Tempelhof - Schöneberg      1066
Treptow - Köpenick           543
Lichtenberg                  463
Steglitz - Zehlendorf        362
Reinickendorf                231
Spandau                      130
Marzahn - Hellersdorf        110
Name: neighbourhood_group, dtype: int64

In [100]:
df_detailed_listings.neighbourhood_group_cleansed.value_counts()

Friedrichshain-Kreuzberg    3516
Mitte                       3426
Pankow                      2480
Neukölln                    2103
Charlottenburg-Wilm.        1287
Tempelhof - Schöneberg      1066
Treptow - Köpenick           543
Lichtenberg                  463
Steglitz - Zehlendorf        362
Reinickendorf                231
Spandau                      130
Marzahn - Hellersdorf        110
Name: neighbourhood_group_cleansed, dtype: int64

The data regarding the 'neighbourhood_group' is also included in the detailed listings csv in the 'neighbourhood_group_cleansed' column. We therefore only need the detailed listings csv, without missing out on features regarding the listings.

In [101]:
print(df_detailed_reviews.info())
print(df_reviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460316 entries, 0 to 460315
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     460316 non-null  int64 
 1   id             460316 non-null  int64 
 2   date           460316 non-null  object
 3   reviewer_id    460316 non-null  int64 
 4   reviewer_name  460316 non-null  object
 5   comments       460293 non-null  object
dtypes: int64(3), object(3)
memory usage: 21.1+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460316 entries, 0 to 460315
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   listing_id  460316 non-null  int64 
 1   date        460316 non-null  object
dtypes: int64(1), object(1)
memory usage: 7.0+ MB
None


Comparing the reviews and detailed reviews csv it becomes clear that the detailed reviews includes all informations necessary. For further analysis we focus on the detailed listings and detailed reviews csv.

In [102]:
print(df_detailed_reviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460316 entries, 0 to 460315
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     460316 non-null  int64 
 1   id             460316 non-null  int64 
 2   date           460316 non-null  object
 3   reviewer_id    460316 non-null  int64 
 4   reviewer_name  460316 non-null  object
 5   comments       460293 non-null  object
dtypes: int64(3), object(3)
memory usage: 21.1+ MB
None


In [103]:
print(df_detailed_listings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15717 entries, 0 to 15716
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            15717 non-null  int64  
 1   listing_url                                   15717 non-null  object 
 2   scrape_id                                     15717 non-null  int64  
 3   last_scraped                                  15717 non-null  object 
 4   source                                        15717 non-null  object 
 5   name                                          15699 non-null  object 
 6   description                                   15435 non-null  object 
 7   neighborhood_overview                         8187 non-null   object 
 8   picture_url                                   15717 non-null  object 
 9   host_id                                       15717 non-null 

Taking a look at the shape and dimensions of the two dataframes shows that there are way more reviews than listings, propably because there are multiple reviews for the same listings included in the review dataframe.

In [104]:
# comparing the entries of both lists and store the ones which aren't present in both to a new list via list comprehension:
# listings_without_reviews = [x for x in df_detailed_listings.id.unique() if x not in df_detailed_reviews.listing_id.unique()]
# print(f'listings without a review: {len(listings_without_reviews)}')

print(f'Unique IDs in the review df: {df_detailed_reviews.listing_id.nunique()}')
print(f'Unique IDs in the listings df: {df_detailed_listings.id.nunique()}')

# to save time computing this method of simply substracting the lists from each other is used. Both methods produce the same output.
print(f'Listings without a review: {df_detailed_listings.id.nunique() - df_detailed_reviews.listing_id.nunique()}')

Unique IDs in the review df: 12755
Unique IDs in the listings df: 15717
Listings without a review: 2962


For further investigation of the data we merge the two dataframes on the respective ID columns.

In [105]:
# renaming the 'id' column into 'review_id':
df_r = df_detailed_reviews.rename(columns={'id': 'review_id'}).copy()

# renaming the 'id' column into 'listing_id':
df_l = df_detailed_listings.rename(columns={'id': 'listing_id'}).copy()

In [111]:
# Doublecheck if everything went well
df_r.review_id.nunique(), df_detailed_reviews.id.nunique(), df_l.listing_id.nunique(), df_detailed_listings.id.nunique()

(460316, 460316, 15717, 15717)

In [114]:
df = pd.merge(df_l, df_r, on='listing_id')

In [117]:
print(f'Shape of df_detailed_reviews: {df_detailed_reviews.shape}')
print(f'Shape of df_detailed_listings: {df_detailed_listings.shape}')
print(f'Shape of the merged dataframe df: {df.shape}')

Shape of df_detailed_reviews: (460316, 6)
Shape of df_detailed_listings: (15717, 75)
Shape of the merged dataframe df: (460316, 80)


In [139]:
# doublecheck if the dataframes are merged correctly, using distinctive columns:
df.groupby(['listing_id'])[['listing_id', 'first_review', 'comments']].head(20)

,listing_id,first_review,comments
0,3176,2009-06-20,"excellent stay, i would highly recommend it. a..."
1,3176,2009-06-20,Britta's apartment in Berlin is in a great are...
2,3176,2009-06-20,"Fantastic, large place in good location. Only ..."
3,3176,2009-06-20,L'appartamento di Britta è molto largo carino ...
4,3176,2009-06-20,We went in Berlin for the new year eve. The ap...
...,...,...,...
460311,765059405014594279,2022-12-06,Wir sind mit dem Flieger erst spät abends ange...
460312,765297203706146084,2022-12-01,Nette und freundliche Leute
460313,765297203706146084,2022-12-01,Eine sehr schöne preiswerte Unterkunft und seh...
460314,765297203706146084,2022-12-01,"Great hosts, thanks :)"


In [142]:
df.to_csv('./resource/airbnb_berlin_data.csv.zip', index_label='listing_id', compression='zip')